<a href="https://colab.research.google.com/github/YaroslavAkhramenko/NLP/blob/main/Copy_of_%D0%92%D0%B0%D1%80%D0%B8%D0%B0%D0%BD%D1%82_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Token classification на примере задачи NER (12 баллов)

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

Классификация токенов — задача, в которой для каждого отдельного токена или слова необходимо определить его тип, например, часть речи. В этом ноутбуке вам предстоит решить подвид задачи Token Classification, а именно NER или Named Entity Recognition. Вам необходимо для каждого слова определить, обозначает ли оно именованную сущность, например, имя человека, название места и тд.

Установим необходимые библиотеки: ```datasets```, ```transformers``` и ```seqeval```.

In [29]:
!pip install datasets
!pip install transformers
!pip install seqeval

In [30]:
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import numpy as np

import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
    )
from datasets import load_dataset, load_metric

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Давайте поближе познакомимся с тем, как хранятся датасеты для NER. В этом задании вам предстоит работать с conll2003. Подробнее о нем можно узнать по этой [ссылке](https://huggingface.co/datasets/conll2003).

В качестве предобученной модели воспользуемся DistilBERT. Это уменьшенная версия обычного BERT.

In [32]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 64

Загрузим данные с помощью функции load_dataset.

In [33]:
datasets = load_dataset("conll2003")

Наши данные состоят из следующих выборок:

In [34]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

В NER существует сразу несколько типов лэйблов для каждого токена. В случае с conll2003 существуют лэйблы следующих видов:

* 'PER' для имен и фамилий
* 'ORG' для названия организаций
* 'LOC' для локаций
* 'MISC' для смешанных сущностей
* 'O' для обычных слов

Также вначале лэйблов бывают буквы B и I. B означает начало сущности, I необходимо для следующего слова, означающего эту же сущность.

In [35]:
label_list = datasets["train"].features[f"ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

Посмотрим на пример из датасета:

In [36]:
example = datasets["train"][4]
print(example.keys())
print(example['tokens'])
print(example['ner_tags'])

dict_keys(['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'])
['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']
[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]


Для каждого отдельного слова есть номер соответствующего лэйбла.

Загрузим токенизатор:

In [37]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Вспомним, что модели семейства BERT используют subword токенизацию, то есть одно слово может получить несколько отдельных токенов.

In [38]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
print("Всего слов:", len(example["tokens"]))

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']
Всего слов: 31


Это означает, что нам необходимо конвертировать лэйблы таким образом, чтобы они соответствовали токенам.

Для того, чтобы проверить к какому слову относится тот или иной токен удобно использовать следующую функцию:

In [39]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


В исходном тексте было 31 слово, столько же индексов выдал и метод ```word_ids()```

### Написание функции для преобразования лэйблов (3 балла)

Ваша задача заключается в том, чтобы написать функцию ```tokenize_and_align_labels()```, которая должна делать токенизацию и преобразовывать лэйблы в формат, соответствующий токенам.

То есть:
* Если слово получило отдельный токен, то ему соответствует один лэйбл
* Если слово получило несколько токенов, то ему должно соответствовать столько же лэйблов. Например, слово crisps получает токенизацию [15594, 2015], тогда в лэйблами для него будет [0, 0]
* Если токен является служебным (имеет индекс None при вызове ```word_ids()```), то ему должен соответствовать лэйбл -100. Это специальный индекс, обозначающий те лэйблы, для которых не нужно считать лосс-функцию.

Пример:

Исходные слова: ```['Only', 'France',
 'and',
 'Britain',
 'backed',
 'Fischler',
 "'s",
 'proposal',
 '.']```

 Исходные лэйблы: ```[0, 5, 0, 5, 0, 1, 0, 0, 0]```

 После токенизации: ```[101, 2069, 2605, 1998, 3725, 6153, 27424, 2818, 3917, 1005, 1055, 6378, 1012, 102]```

 Измененные лэйблы: ```[-100, 0, 5, 0, 5, 0, 1, 1, 1, 0, 0, 0, 0, -100]```

 Также дополнительные примеры можно посмотреть в следующих ячейках ноутбука, которые проверяют корректность реализации функции ```tokenize_and_align_labels()```

In [40]:
def tokenize_and_align_labels(examples):
    print(examples)
    # Токенизируем текст
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []  # В этот массив будем складывать id лэйблов токенов
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        # Напишите код здесь. Соберите в список label_ids лэйблы, соответствующие токенам
        label_ids = []
        for j in word_ids:
            if j is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[j])

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [41]:
test_examples = {
    'id': ['0', '1', '2'],
    'tokens': [
        ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
        ['Peter', 'Blackburn'],
        ['BRUSSELS', '1996-08-22']
        ],
    'ner_tags': [
        [3, 0, 7, 0, 0, 0, 7, 0, 0],
        [1, 2],
        [5, 0]
        ]
    }

test_outputs = {
    'input_ids': [
        [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102],
        [101, 2848, 13934, 102],
        [101, 9371, 2727, 1011, 5511, 1011, 2570, 102]
        ],
    'attention_mask': [
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]
        ],
    'labels': [
        [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100],
        [-100, 1, 2, -100],
        [-100, 5, 0, 0, 0, 0, 0, -100]
        ]
    }

In [42]:
assert test_outputs == tokenize_and_align_labels(test_examples), "Похоже tokenize_and_align_labels работает не так, как должна"

{'id': ['0', '1', '2'], 'tokens': [['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], ['Peter', 'Blackburn'], ['BRUSSELS', '1996-08-22']], 'ner_tags': [[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2], [5, 0]]}


Применим функцию ко всем выборкам датасета с помощью метода ```map()```

In [43]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

## Тренировака модели


Обучать модель будем с помощью ```Trainer``` из библиотеки ```transformers```.

Загрузим претренированные веса:

In [44]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Определение аргументов для тренировки (1 балл)

Загляните в [документацию](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) и заполните необходимые аргументы для тренировки. Помните, что для файнтюнинга больших моделей следует выбирать небольшой learning rate (обычно меньше 1е-5).

In [45]:
!pip show accelerate

Name: accelerate
Version: 0.21.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, packaging, psutil, pyyaml, torch
Required-by: 


In [46]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    output_dir='./res',
    evaluation_strategy='epoch',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
)

Создадим вспомогательные объекты: ```DataCollatorForTokenClassification``` и ```metric```

In [47]:
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

### Расчет метрики (1 балл)

Опишем функцию ```compute_metric```, которая будет учитывать только нужные токены.

In [48]:
import numpy as np

print(label_list)

def compute_metrics(predictions, labels):
    predictions = np.argmax(predictions, axis=2)

    # Удалим из подсчета метрик служебные токены

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # По аналогии с фильтрацией true_predictions опишите фильтрацию для true_labels
    true_labels =  [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]# магия здесь

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


### Использование ```Trainer``` для обучения (2 балла)

Далее создайте объект класса ```Trainer``` с необходимыми аргументами и обучите модель.

Подробнее о том, как использовать ```Trainer```, можно почитать [здесь](https://huggingface.co/docs/transformers/main_classes/trainer) или же посмотреть семинарское занятия из этого модуля.  

In [49]:
# Создайте объект класса Trainer и обучите модель
#print(datasets['train'])

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer = tokenizer,
    data_collator = data_collator
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.348800,0.124168
2,0.072100,0.112804
3,0.049900,0.119309
4,0.039000,0.122487


TrainOutput(global_step=3512, training_loss=0.10191591927497425, metrics={'train_runtime': 352.3106, 'train_samples_per_second': 159.416, 'train_steps_per_second': 9.968, 'total_flos': 682592149798524.0, 'train_loss': 0.10191591927497425, 'epoch': 4.0})

### Получение необходимой метрики (3 балла)

 Хорошее качество для этой задачи ~0.92 по F1 мере или выше. Попробуйте добиться этого значения, используя различные гиперпараметры в ```TrainingArguments```. Напишите вывод о проделанной работе.

In [50]:
# Код
compute_metrics(trainer.predict(tokenized_datasets['validation']).predictions, tokenized_datasets['validation']['labels'])
#Я менял
#1) batch_size - оптимальный 16,
#2) learning_rate - при 1е-6 получилось 0.89. При 4е-5 получилось 0.93758
#Исходя из затраченого времени, количество эпох более 4х не ставилось

{'precision': 0.9152355790400705,
 'recall': 0.9300816646157288,
 'f1': 0.922598901403762,
 'accuracy': 0.9821595945795669}

### Дополнительный эксперимент (2 балла)

А теперь попробуйте решить ту же задачу, но с другой претренированной моделью из семейства BERT, например, ```roberta-base``` или ```distillroberta-base``` и получить качество выше 0.94 по F1 на валидационном датасете. Список доступных моделей можно посмотреть [здесь](https://huggingface.co/models). Вы на практике убедитесь, насколько различные претренированные модели могут улучшать конечное качество на downstream задачах.

Для выполнения этого пункта можно всего лишь скопировать некоторые ячейки кода выше и поменять переменную ```model_checkpoint``` на название другой модели.

In [51]:
EXargs = TrainingArguments(
    output_dir='./res',
    evaluation_strategy='epoch',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
)

In [52]:
# Проведите эксперимент здесь

EXtrainer = Trainer(
    model='dslim/bert-base-NER',
    args=EXargs,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer = tokenizer,
    data_collator = data_collator
)

EXtrainer.train()

AttributeError: ignored

In [ ]:
compute_metrics(EXtrainer.predict(tokenized_datasets['validation']).predictions, tokenized_datasets['validation']['labels'])